Many codes from the orginal implementation https://github.com/ultralytics/yolov5 (GPL-3 licence) but with some adjustments to fit the competition and Pseudo Labeling

Discussion topic to the notebook https://www.kaggle.com/c/nfl-impact-detection/discussion/206832

In [ ]:
!pip install ../input/seaborn0111/seaborn-0.11.1-py3-none-any.whl

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm.auto import tqdm
import torch
import cv2
from glob import glob

In [ ]:
!cp -r ../input/yololatest/yolov5-master/* .

In [ ]:
from utils.datasets import LoadImages
from models.experimental import attempt_load
from utils.torch_utils import time_synchronized
from utils.general import *
from utils.plots import plot_one_box
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, \
    strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

import sys
sys.path.insert(0, "timm-efficientdet-pytorch")
sys.path.insert(0, "omegaconf")

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
#from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import pandas as pd
#from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
#from effdet.efficientdet import HeadNet
from tqdm import tqdm, tqdm_notebook
#from torch.cuda.amp import autocast,GradScaler

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = False

seed_everything(SEED)

In [ ]:
d = pd.read_csv('../input/nfl-impact-detection/test_player_tracking.csv')
IS_PRIVATE = d.shape != (19269, 12)
print(IS_PRIVATE)

ISTEST = False

In [ ]:
if ISTEST:
    IS_PRIVATE = True

In [ ]:
!mkdir /tmp/sets

In [ ]:
##https://github.com/ultralytics/yolov5

%%writefile models/yolov5x.yaml

# parameters
nc: 1  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]


In [ ]:
##https://github.com/ultralytics/yolov5
%%writefile models/nfl.yaml

train: /tmp/sets/convertor/fold0/images/train2017/
val: /tmp/sets/convertor/fold0/images/val2017/

# number of classes
nc: 1

# class names
names: ['Helmet']

In [ ]:
df = pd.read_csv('../input/trainnoxframes5fold/video_labels.csv')
df["image_name"] = df["image_name"].str.replace(".png", "")
#df = df.drop('Unnamed: 0', axis=1)
#df = df[df['impact'] > 1]
#bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
#for i, column in enumerate(['x', 'y', 'w', 'h']):
#    df[column] = bboxs[:,i]
#df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
df['image_id'] = df['image_name']
from tqdm.auto import tqdm
import shutil as sh
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
df.reset_index(inplace=True)
df = df.drop('index',axis=1)
df

In [ ]:
if IS_PRIVATE:
    !cp -r ../input/trainnoxframes5fold/convertor /tmp/sets


In [ ]:
##https://github.com/ultralytics/yolov5
if IS_PRIVATE:
    ImagesizePred = 1280
    WEIGHTS = ['../input/yolo5recfulls2/last (1).pt','../input/yoloxframes144epochs/best (2).pt','../input/yolo5realxframes1280rectrain44e/last.pt']

    sourcedatas = '../input/nfl-impact-detection/test'

    source, weights, view_img, save_txt, imgsz = sourcedatas, WEIGHTS, False , True , ImagesizePred
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
            ('rtsp://', 'rtmp://', 'http://'))
    projectsave = ''
    projname = 'exp'
    deviceopt = ''
    existok = False
    augmentopt = False
    conf_thres = 0.5
    iou_thres = 0.5
    USESTH = False
    score_thres = 0.35
    saveconf = False
    result_image_ids = []
    results_boxes = []
    results_scores = []

    save_dir = Path(increment_path(Path(projectsave) / projname, exist_ok=existok))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    set_logging()
    device = select_device(deviceopt)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

        # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

        # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz)

        # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=augmentopt)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)
        t2 = time_synchronized()

            # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)
        image_id = Path(path).name
        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path

            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            image_id = str(p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                result_image_ids += [image_id]*len(det)
                results_boxes.append(det[:, :4].cpu().data.numpy())
                results_scores.append(det[:, 4].cpu().data.numpy())

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f'{n} {names[int(c)]}s, '  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if saveconf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or view_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fourcc = 'mp4v'  # output video codec
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    print(f'Done. ({time.time() - t0:.3f}s)')

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
if IS_PRIVATE:
    DATA_ROOT_PATH = '/tmp/sets/convertor/fold0/images/train2017'
    # https://www.kaggle.com/its7171/2class-object-detection-inference/
    def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
        video_path=f"{video_dir}/{video_name}"
        video_name = os.path.basename(video_path)
        vidcap = cv2.VideoCapture(video_path)
        if only_with_impact:
            boxes_all = video_labels.query("video == @video_name")
            print(video_path, boxes_all[boxes_all.impact > 0].shape[0])
        else:
            print(video_path)
        frame = 0
        while True:
            it_worked, img = vidcap.read()
            if not it_worked:
                break
            frame += 1
            if only_with_impact:
                boxes = video_labels.query("video == @video_name and frame == @frame")
                boxes_with_impact = boxes[boxes.impact > 0]
                if boxes_with_impact.shape[0] == 0:
                    continue
            img_name = f"{video_name}_frame{frame}"
            image_path = f'{out_dir}/{video_name}'.replace('.mp4',f'_{frame}.png')
        #    dim = (1024, 1024)
         #   img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            _ = cv2.imwrite(image_path, img)

    #if TESTMODE:
    out_dir = DATA_ROOT_PATH
    !mkdir -p $out_dir
    video_dir = '/kaggle/input/nfl-impact-detection/test'
    uniq_video = [path.split('/')[-1] for path in glob(f'{video_dir}/*.mp4')]
    for video_name in uniq_video:
        mk_images(video_name, pd.DataFrame(), video_dir, out_dir, only_with_impact=False)
        # for fast commit
    #    if len(uniq_video) == 6:
      #      break


In [ ]:
if IS_PRIVATE:
    !cp exp/labels/*.* /tmp/sets/convertor/fold0/labels/train2017/

In [ ]:
##https://github.com/ultralytics/yolov5
if IS_PRIVATE:
    !WANDB_MODE="dryrun" python train.py --rect --img-size 1280 --batch-size 4 --epochs 206 --data models/nfl.yaml --cfg models/yolov5x.yaml --weights '../input/yolo5recfulls2/last (1).pt'

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
##https://github.com/ultralytics/yolov5
if IS_PRIVATE:
    ImagesizePred = 1280
    WEIGHTS = ['runs/train/exp/weights/last.pt','../input/yolo5recfulls2/last (1).pt','../input/yoloxframes144epochs/best (2).pt','../input/yolo5realxframes1280rectrain44e/last.pt']

    sourcedatas = '../input/nfl-impact-detection/test'

    source, weights, view_img, save_txt, imgsz = sourcedatas, WEIGHTS, False , False , ImagesizePred
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
            ('rtsp://', 'rtmp://', 'http://'))
    projectsave = ''
    projname = 'exp'
    deviceopt = ''
    existok = False
    augmentopt = False
    conf_thres = 0.5
    iou_thres = 0.5
    USESTH = False
    score_thres = 0.35
    result_image_ids = []
    results_boxes = []
    results_scores = []

    save_dir = Path(increment_path(Path(projectsave) / projname, exist_ok=existok))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    set_logging()
    device = select_device(deviceopt)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

        # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

        # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz)

        # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=augmentopt)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes=None, agnostic=False)
        t2 = time_synchronized()

            # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)
        image_id = Path(path).name
        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path

            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            image_id = str(p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                det[:, 2] = det[:, 2] - det[:, 0]
                det[:, 3] = det[:, 3] - det[:, 1]
                det[:, 0] = det[:, 0].clip(min=0, max=1280-1)
                det[:, 2] = det[:, 2].clip(min=0, max=1280-1)
                det[:, 1] = det[:, 1].clip(min=0, max=720-1)
                det[:, 3] = det[:, 3].clip(min=0, max=720-1)

                result_image_ids += [image_id]*len(det)
                results_boxes.append(det[:, :4].cpu().data.numpy())
                results_scores.append(det[:, 4].cpu().data.numpy())

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f'{n} {names[int(c)]}s, '  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or view_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fourcc = 'mp4v'  # output video codec
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    print(f'Done. ({time.time() - t0:.3f}s)')

In [ ]:
if IS_PRIVATE:
    len(results_boxes)

In [ ]:
if IS_PRIVATE:
    box_df = pd.DataFrame(np.concatenate(results_boxes), columns=['left', 'top', 'width', 'height'])
    test_df = pd.DataFrame({'scores':np.concatenate(results_scores), 'image_name':result_image_ids})
    test_df = pd.concat([test_df, box_df], axis=1)

    if USESTH:
        test_df = test_df[test_df.scores > score_thres]

    test_df.shape

In [ ]:
if IS_PRIVATE:
    test_df

In [ ]:
if IS_PRIVATE:
    #gameKey,playID,view,video,frame,left,width,top,height
    #57590,3607,Endzone,57590_003607_Endzone.mp4,1,1,1,1,1
    test_df['gameKey'] = test_df.image_name.str.split('_').str[0].astype(int)
    test_df['playID'] = test_df.image_name.str.split('_').str[1].astype(int)
    test_df['view'] = test_df.image_name.str.split('_').str[2]
    test_df['frame'] = test_df.image_name.str.split('_').str[3].str.replace('.png','').astype(int)
    test_df['video'] = test_df.image_name.str.rsplit('_',1).str[0] + '.mp4'
    test_df = test_df[["gameKey","playID","view","video","frame","left","width","top","height"]]
    test_df

In [ ]:
!mv * /tmp/

In [ ]:
import nflimpact
env = nflimpact.make_env()

if IS_PRIVATE:
    env.predict(test_df) # df is a pandas dataframe of your entire submission file
else:
    sub = pd.read_csv('../input/nfl-impact-detection/sample_submission.csv')
    env.predict(sub)